# 인공지능을 사용한 뇌 MRI 이미지 분류
## 끈적이는 정도를 level 1 ~ 4까지로 피실험자의 visual / touch / auditory 3가지 방법을 통해 CNN 이미지 인공지능을 사용하여 뇌의 MRI 이미지를 분류한 프로젝트 
### - 작성자 : 권민호 / 프로젝트 인원 : 2명
#### - 작성 일자 : 2022/09/29 , 프로젝트 일자 : 2022/09/15 ~ 

In [1]:
import pandas as pd
import os
import shutil

# import splitfolders # train folder split
# splitfolders.ratio('./MRI_nii/touch/',output='./touch_train',seed=2304,ratio=(0.7,0.3))

Copying files: 2304 files [01:19, 29.03 files/s] 


## .csv file 전처리

In [3]:
# subj : 피실험자 번호(2~27, 21,22 없음) / trial : 시행 횟수 / level : 끈적임 레벨 / modality : 자극 방식(시각 / 촉각 / 청각) / orders : 반복 횟수
# 피실험자 당 실험 횟수 : (반복횟수) * (시행 횟수) * (자극 방식) --> 2 * 48 * 3 = 288
# 총 실험 횟수 : (피실험자 당 실험 횟수) * (피실험자 수) --> 288 * 24 = 6912

df = pd.read_csv('./안쓰는 파일/bhv_data_AllinOne.csv',index_col=0)
df = pd.DataFrame(df)
df = df.drop(columns={'resp','correct','block'})

modality_list = list(df['modality'].unique())
level_list = list(df['level'].unique())
df

,subj,trial,level,modality,orders
0,2,1,3,touch,1
1,2,2,4,touch,1
2,2,3,3,touch,1
3,2,4,1,touch,1
4,2,5,4,touch,1
...,...,...,...,...,...
6907,27,44,2,auditory,2
6908,27,45,1,auditory,2
6909,27,46,4,auditory,2
6910,27,47,4,auditory,2


### 자극 modality 별 level 분류하여 폴더에 저장

In [ ]:
# for modal in modality_list: # ['visual','auditory','touch']
#     os.mkdir(f'./MRI_train/{modal}')
#     for level in level_list: # [1, 2, 3, 4]
#         new_df = df.loc[(df['modality'] == modal) & (df['level'] == level)] # 'visual & level 1~4'
#         new_df.to_csv(f'./MRI_train/{modal}/level_{level}.csv')

### EX) 002_auditory_01_trial_0001_correct_beta.nii

- [002 ~ 027(21,22 제외)] subj
- [touch,visual,auditory] modality
- [01,02] orders
- [0001~0055] trial
- [correct,incorrect] 
- [beta,nonbeta].nii 

csv 파일의 subj 의 trial로 .nii 파일의 level을 매칭할 수 있음

In [8]:
# 피실험자 subj 넘버 리스트 생성
subj_list_under_10 = ['00'+str(i) for i in range(2,10)]
subj_list_over_10 = ['0'+ str(i) for i in range(10,28) if i not in [21,22]]
subj_list = subj_list_under_10 + subj_list_over_10

# 자극 방식 리스트
modality_list

# 반복 횟수 리스트
orders_list = ['01','02']

# 시도횟수 리스트 
trial_list_under_10 = ['000'+str(i) for i in range(1,10)]
trial_list_over_10 = ['00'+str(i) for i in range(10,49)]
trial_list = trial_list_under_10 + trial_list_over_10

# correct 리스트
correct_list = ['correct','incorrect']

# .nii 파일 리스트
file_path = './raw_beta_map/'
file_name_list = os.listdir(file_path)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27]


### .nii 파일 분류

In [44]:
# subj + '_' + modality + '_' + orders + '_' + 'trial' + '_' + trial + '_' + correct + '_' + 'beta.nii'
src = './raw_beta_map/'
for subj in subj_list: # 002_
    for modality in modality_list: # 002_auditory_
        dir = f'./MRI_nii/{modality}/'
        for orders in orders_list: # 002_auditory_01_
            for trial in trial_list: # 002_auditory_01_trial_0001_
                for correct in correct_list: # 002_auditory_01_trial_0001_correct_
                    file_name = subj + '_' + modality + '_' + orders + '_' + 'trial' + '_' + trial + '_' + correct + '_' + 'beta.nii'
                    new_df = df[(df['subj']==int(subj)) & (df['trial']==int(trial)) & (df['orders'] == int(orders)) & (df['modality'] == modality)]
                    level = new_df['level'].values[0]
                    
                    if file_name in file_name_list: # 예외 처리
                        shutil.move(src+file_name, dir+f'level_{level}/'+file_name)